<a href="https://colab.research.google.com/github/jonesavelino/idea-c2-tool/blob/main/IDEA_ETAPA_1_Pre_Anotacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETAPA 1 - PRÉ-ANOTAÇÃO DE TEXTOS DE C2

## Caderno resposável por pré-anotar os textos do corpus e interoperar com a ferramenta Doccano. De acordo com o arquivo doutrinário de entrada, utilizando o metamodelo IDEA-C2, é possível pré-anotar entidades e relações alinhada à abordagem supervisionada à distância por meio de expressões regulares. Como resultado, o arquivo JSON gerado contém os termos anotados para ser importado pela ferramenta Doccano.

### Passo a passo
#### 1 - "Setar" o caminho /content/drive/MyDrive/Doutorado/IDEA/importa_glossario/
##### 1.1 - Arquivos de Entrada (Glossário): /content/drive/MyDrive/Doutorado/IDEA/importa_glossario/docs/*.pdf (C-20-1- Glossário do EB.pdf)
##### 1.2 - Arquivos de Saída (1): /content/drive/MyDrive/Doutorado/IDEA/importa_glossario/texts/output.txt
##### 1.3 - Arquivos de Saída (2) - JSONL: /content/drive/MyDrive/Doutorado/IDEA/importa_glossario/texts/exposta*.jsol
##### 1.4 - Observação: O arquivo de saída JSONL deve ser importado no Doccano.


# Data de Criação: 01-10-2023
# Última atualização: 20-11-2023
# Autores: Gustavo Danon e Jones.

In [ ]:
# Monta o drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [ ]:
# Setando a pasta onde estão localizados os outros arquivos .py que auxiliam a execução
import sys
sys.path.insert(0,'/content/drive/MyDrive/Doutorado/IDEA/importa_glossario/')
import os
# # tem que setar o diretório
os.chdir('/content/drive/MyDrive/Doutorado/IDEA/importa_glossario/')
cwd = os.getcwd()
print(cwd)

'''
O notebook está especificado para cada regex, então mudar na mao os regex de:
    page_regex : as quebras de página do arquivo (indice da pagina ou ordenacao do glossario)
    term_regex : o regex para extrair os sujeitos
    delimiter_regex : o regex para substituir pelo predicado
Mudar tambem page_start e page_end para pular o glossario e o fim do arquivo
'''

import pdf_functions
import terms_functions
from find_all import *
import enrich_relation as ER

import re
import sys
import json

import math

/content/drive/MyDrive/Doutorado/IDEA/importa_glossario


In [ ]:
# Função para retirar caracteres indesejáveis
def limpar_texto(text):
  # text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
  # text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
  # text = re.sub(r"[^a-zA-Z.!?']", ' ', text)
  # text = re.sub(r" +", ' ', text)
  #text = re.sub(".", "", text)
  text = re.sub("/", " ", text)
  text = re.sub("[()]/", " ", text)
  #text = text.lower()
  return text

In [ ]:
# File info
pdf_path = 'docs/C-20-1- Glossário do EB.pdf'

pdf_text = pdf_functions.pdf_to_text(pdf_path, page_start=12, page_end=363)
raw_lines = pdf_text.split('\n')

for i in range(len(raw_lines)):
  raw_lines[i] = limpar_texto(raw_lines[i])
  #raw_lines[i] = raw_lines[i]

print(raw_lines)

# Change page regex here to format them correctly
#**********************************************************************************
page_regex = r'\A[A-ZÀ-Ú][-–](.*)$|\A[A-ZÀ-Ú]$|\A[A-ZÀ-Ú] ([0-9]*)-([0-9]*)$'
page_regex = re.compile(page_regex)

lines = pdf_functions.filter_page_number(raw_lines, page_regex)

delimiter_regex = r'[-–˗⁻−﹣](\s*)((1\.\s)?)[A-ZÀ-Ú][a-zà-ú ]'
term_regex = r'^[A-ZÀ-ÖØ-Þ\(\)\s\/,-]{2,}'
#**********************************************************************************

['C 20-1', 'A', 'ABALROAMENTO – Choque entre duas ou mais - ou de uma com duas ou', 'mais - aeronaves, em vôo ou em manobra na superfície, ou embarcações de', 'qualquer espécie, em porto, local de atracação, ou via navegável, e do qual', 'resultem danos.', 'ABASTECIMENTO – Conjunto de atividades que tem o propósito de prever', 'e prover, para as forças e organizações militares, o material necessário a mantê-', 'las em condições de plena eficiência. Assim, o abastecimento proporciona um', 'fluxo adequado do material necessário, desde as fontes de obtenção até as', 'organizações militares consumidoras, abrangendo a função logística suprimento', 'e parte da função logística transporte, além de relacionar-se, estreitamente, com', 'a função logística manutenção. Ver FUNÇÃO LOGÍSTICA SUPRIMENTO.', 'ABICAR – Manobra realizada por navios ou embarcações, encalhando', 'intencionalmente com a proa em uma praia ou margem, de maneira controlada,', 'com o fim de desembarcar pessoal ou material.', 'A

In [ ]:
#print(len(terms))

## Estratégia de Geração do JSONL - 1
### Código que implementa a geração do arquivo .JSONL, fixando um termo por documento
### Depois que rodar esse script, tem que rodar o outro script de ajuste dos IDs

In [ ]:
JSONL_COMPLETO=[]
FIRST_TERM = 0
TERMOS_DIVISAO = 1  #  Variável que indica a quantidade de termos o usuário quer fatiar por documento
LAST_TERM = TERMOS_DIVISAO
TOTAL_TERMOS = 3294
TOTAL_DIVISAO=math.floor(TOTAL_TERMOS/TERMOS_DIVISAO)
#TOTAL_TERMOS = 3294
#TOTAL_DIVISAO=math.floor(TOTAL_TERMOS/TERMOS_DIVISAO)
i=0
while i<TOTAL_DIVISAO:
#while i<=TOTAL_TERMOS:
    #print('Processando dados de ' + str(i) +  ' a ' + str(TOTAL_DIVISAO) + ' - Termos de: ' + str(FIRST_TERM) + ' a ' + str(LAST_TERM))

    terms, all_formated_lines = terms_functions.terms_and_lines(lines, term_regex, delimiter_regex)
    terms = list(terms.values())

    formated_lines = all_formated_lines[FIRST_TERM:LAST_TERM]

    #print(terms)
    #print(len(terms))

    # raw_file = 'texts/raw.txt'
    # with open(raw_file, 'w') as output:
    #   output.write('\n'.join(lines))

    # output_file = 'texts/formated_lines.txt'
    # with open(output_file, 'w') as output:
    #   output.write('\n'.join(formated_lines))

    relation_terms = find_relation_terms(**{
        'lines': formated_lines,
        'terms': terms,
        'term_index': max(tf.get_ids(terms)) + 1,
        'slice_term_index': FIRST_TERM
    })

    main_relations = find_main_relations(**{
      'lines': formated_lines,
      'terms': terms,
      'relation_terms': relation_terms,
      'relation_type' : 'associated_with', #tive que inserir isso aqui pra ele aceitar o parâmetro com o novo valor
      'slice_term_index': FIRST_TERM
    })

    relation_models = [
          ER.Relations('. Ver ', 'associated_with'),
          ER.Relations('. O mesmo que ', 'equivalent_to'),
          ER.Relations(' tipo de ', 'type_of'),
          ER.Relations(' subunidade ', 'type_of'),
          ER.Relations(' executado p ', 'responsible_for'),
          ER.Relations(' responsável p ', 'responsible_for'),
          ER.Relations(' designad ', 'responsible_for'),
          ER.Relations(' composto p ', 'composed_of'),
          ER.Relations(' conjunto d ', 'composed_of'),
          ER.Relations(' capacidade d ', 'capacity_of'),
          ER.Relations(' ocorr ', 'occurs_in'),
          ER.Relations(' emprega ', 'applied_to'),
          ER.Relations(' aplica ', 'applied_to')
    ]

    jsonString = {
          "id": i,
          "text": ' '.join(formated_lines)
    }

    ER.enrich_relations('\n'.join(all_formated_lines), relation_models, main_relations)

    # Ajustar o offset para fatiar p texto
    first_offset = terms[FIRST_TERM]['start_offset']
    terms_functions.shift_terms(terms, first_offset)
    terms_functions.shift_terms(relation_terms, first_offset)

    jsonRelations = {"relations": []}
    all_terms = list()

    jsonRelations['relations'].extend(main_relations)

    jsonEntities = {
        "entities": list()
    }

    jsonEntities['entities'].extend(terms[FIRST_TERM:LAST_TERM])
    jsonEntities['entities'].extend(relation_terms)

    jsonString.update(jsonEntities)
    jsonString.update(jsonRelations)
    jsonString.update({"Comments": []})

    # Escrita dos JSON
    #jsonFile = 'texts/output.jsonl'
    jsonFile = ''
    jsonFile = 'texts/output_' + str(i) + '-' + str(FIRST_TERM) + '_a_' + str(LAST_TERM) + '.jsonl'

    # linha de geração do arquivo em disco fatiado foi comentada - isso tá funcionando
    # with open(jsonFile, 'w', encoding='utf-8') as json_file:
    #     json.dump(jsonString, json_file, ensure_ascii=False)
    #print(f"Resultados salvos em {jsonFile}")

    if jsonString != '':
      JSONL_COMPLETO.append(jsonString)

    # FIRST_TERM = LAST_TERM + 1
    # LAST_TERM = LAST_TERM + TERMOS_DIVISAO

    FIRST_TERM = LAST_TERM
    LAST_TERM = LAST_TERM + 1

    if LAST_TERM > TOTAL_TERMOS:
      LAST_TERM = TOTAL_TERMOS
      j=TOTAL_DIVISAO
    i=i+1
#  Fim do While
if JSONL_COMPLETO != '':
  jsonFile = 'texts/output_' + str(i) + '-' + str(FIRST_TERM) + '_a_' + str(LAST_TERM) + ' - COMPLETO.jsonl'
  #with open(jsonFile, 'w', encoding='utf-8') as json_file:
  #      json.dump(JSONL_COMPLETO, json_file, ensure_ascii=False)
  with open(jsonFile, 'w', encoding='utf-8') as json_file:
    for ddict in JSONL_COMPLETO:
        jout = json.dumps(ddict) + '\n'
        json_file.write(jout)
  print(f"Resultados salvos em {jsonFile}")



Resultados salvos em texts/output_3294-3294_a_3294 - COMPLETO.jsonl


## Script complementar e fundamental para a geração do JSONL responsável por ajustar os IDs

In [ ]:
data_file_path = 'texts/output_66-3295_a_3294 - COMPLETO.jsonl'
DATA_FILE_PATH_JSON_OUT = 'texts/output_66-3295_a_3294 - COMPLETO-modificado.jsonl'

cont_doc = 0
cont_ent = 0
cont_rel = 0
json_list = []
map_ent = {}

with open(data_file_path, 'r', encoding='utf-8') as jsonl_file:
    json_total_list = []
    num_lines = 0

    for line in jsonl_file:
        json_total_list.append(json.loads(line))
        num_lines += 1

print(json_total_list)

#
for cont_doc, docum in enumerate(json_total_list):
    lista_entidades = []
    lista_relations = []
    text = docum['text']
    entities =  docum['entities']
    relations =  docum['relations']
    for ent in entities:
        print(ent)
        ent_id = ent['id']
        ent_start = ent['start_offset']
        ent_end = ent['end_offset']
        ent_label = ent['label']
        print('(', ent_start, '-', ent_end, ')', '-', ent_label)

        dict_entidades = {
            "id": cont_ent,
            "label": ent_label,
            "start_offset": ent_start,
            "end_offset": ent_end
        }

        lista_entidades.append(dict_entidades)
        map_ent[ent_id] = cont_ent
        cont_ent += 1

    for rel in relations:  # "id": 1740, "from_id": 82887, "to_id": 82885, "type": "usado_para"
        print(rel)
        rel_id = rel['id']
        rel_from_id = map_ent[rel['from_id']]
        rel_to_id = map_ent[rel['to_id']]
        rel_type = rel['type']
        print('(', rel_from_id, '-', rel_to_id, ')', '-', rel_type)

        dict_relations = {
            "id": cont_rel,
            "from_id": rel_from_id,
            "to_id": rel_to_id,
            "type": rel_type
        }

        lista_relations.append(dict_relations)
        cont_rel += 1

    dict_anotacoes = {
        "id": cont_doc,
        "text": text,
        "entities": lista_entidades,
        "relations": lista_relations,
        "Comments": []
    }

    json_list.append(dict_anotacoes)

with open(DATA_FILE_PATH_JSON_OUT, 'w') as f_saida:
    for ddict in json_list:
        jout = json.dumps(ddict,ensure_ascii=False) + '\n'
        f_saida.write(jout)

A saída de streaming foi truncada nas últimas 5000 linhas.
{'id': 6714, 'label': 'entity', 'start_offset': 0, 'end_offset': 36, 'texto_key': 'MANUTENÇÃO CORRETIVA NÃO-PROGRAMADA'}
( 0 - 36 ) - entity
{'id': 6716, 'label': 'entity', 'start_offset': 78, 'end_offset': 109, 'texto_key': 'MANUTENÇÃO CORRETIVA PLANEJADA'}
( 78 - 109 ) - entity
{'id': 6723, 'label': 'entity', 'start_offset': 491, 'end_offset': 526, 'texto_key': 'MANUTENÇÃO DA FISIONOMIA DA FRENTE'}
( 491 - 526 ) - entity
{'id': 6727, 'label': 'entity', 'start_offset': 722, 'end_offset': 750, 'texto_key': 'MANUTENÇÃO DA ORDEM PÚBLICA'}
( 722 - 750 ) - entity
{'id': 6731, 'label': 'entity', 'start_offset': 985, 'end_offset': 1003, 'texto_key': 'MANUTENÇÃO DA PAZ'}
( 985 - 1003 ) - entity
{'id': 6754, 'label': 'entity', 'start_offset': 2510, 'end_offset': 2533, 'texto_key': 'MANUTENÇÃO DE CAMPANHA'}
( 2510 - 2533 ) - entity
{'id': 6759, 'label': 'entity', 'start_offset': 2808, 'end_offset': 2833, 'texto_key': 'MANUTENÇÃO DE NÍVE

In [ ]:
import math
FIRST_TERM = 0
TERMOS_DIVISAO = 2
LAST_TERM = TERMOS_DIVISAO
TOTAL_TERMOS = 11
TOTAL_DIVISAO=math.floor(TOTAL_TERMOS/TERMOS_DIVISAO)
i=0
while i<=TOTAL_DIVISAO:
    print('Processando dados de ' + str(i) +  ' a ' + str(TOTAL_DIVISAO) + ' - Termos de: ' + str(FIRST_TERM) + ' a ' + str(LAST_TERM))
    FIRST_TERM = LAST_TERM + 1
    LAST_TERM = LAST_TERM + TERMOS_DIVISAO

    if LAST_TERM > TOTAL_TERMOS:
      LAST_TERM = TOTAL_TERMOS
      j=TOTAL_DIVISAO
    i=i+1




Processando dados de 0 a 5 - Termos de: 0 a 2
Processando dados de 1 a 5 - Termos de: 3 a 4
Processando dados de 2 a 5 - Termos de: 5 a 6
Processando dados de 3 a 5 - Termos de: 7 a 8
Processando dados de 4 a 5 - Termos de: 9 a 10
Processando dados de 5 a 5 - Termos de: 11 a 11


## Estratégia de Geração do JSONL - 2
### Código que implementa a geração do arquivo .JSONL de acordo com as fatias de texto determinadas
### Depois que rodar esse script, tem que rodar o outro script de ajuste dos IDs

In [ ]:
JSONL_COMPLETO=[]
FIRST_TERM = 0
TERMOS_DIVISAO = 50  #  Variável que indica a quantidade de termos o usuário quer fatiar por documento
LAST_TERM = TERMOS_DIVISAO
TOTAL_TERMOS = 3294
#TOTAL_TERMOS = 3294
TOTAL_DIVISAO=math.floor(TOTAL_TERMOS/TERMOS_DIVISAO)
i=0

# raw_file = 'texts/raw.txt'
# with open(raw_file, 'w') as output:
#   output.write('\n'.join(lines))

# output_file = 'texts/formated_lines.txt'
# with open(output_file, 'w') as output:
#   output.write('\n'.join(formated_lines))

while i<=TOTAL_DIVISAO:
  #print('Processando dados de ' + str(i) +  ' a ' + str(TOTAL_DIVISAO) + ' - Termos de: ' + str(FIRST_TERM) + ' a ' + str(LAST_TERM))

  terms, all_formated_lines = terms_functions.terms_and_lines(lines, term_regex, delimiter_regex)
  terms = list(terms.values())

  formated_lines = all_formated_lines[FIRST_TERM:LAST_TERM]

  #print(terms)
  #print(len(terms))

  relation_terms = find_relation_terms(**{
      'lines': formated_lines,
      'terms': terms,
      'term_index': max(tf.get_ids(terms)) + 1,
      'slice_term_index': FIRST_TERM
  })

  main_relations = find_main_relations(**{
    'lines': formated_lines,
    'terms': terms,
    'relation_terms': relation_terms,
    'relation_type' : 'associated_with', #tive que inserir isso aqui pra ele aceitar o parâmetro com o novo valor
    'slice_term_index': FIRST_TERM
  })

  relation_models = [
        ER.Relations('. Ver ', 'associated_with'),
        ER.Relations('. O mesmo que ', 'equivalent_to'),
        ER.Relations(' tipo de ', 'type_of'),
        ER.Relations(' subunidade ', 'type_of'),
        ER.Relations(' executado p ', 'responsible_for'),
        ER.Relations(' responsável p ', 'responsible_for'),
        ER.Relations(' designad ', 'responsible_for'),
        ER.Relations(' composto p ', 'composed_of'),
        ER.Relations(' conjunto d ', 'composed_of'),
        ER.Relations(' capacidade d ', 'capacity_of'),
        ER.Relations(' ocorr ', 'occurs_in'),
        ER.Relations(' emprega ', 'applied_to'),
        ER.Relations(' aplica ', 'applied_to')
  ]

  #print(relation_models)

  jsonString = {
        "id": i,
        "text": ' '.join(formated_lines)
  }

  ER.enrich_relations('\n'.join(all_formated_lines), relation_models, main_relations)

  # Ajustar o offset para fatiar p texto
  first_offset = terms[FIRST_TERM]['start_offset']
  terms_functions.shift_terms(terms, first_offset)
  terms_functions.shift_terms(relation_terms, first_offset)

  jsonRelations = {"relations": []}
  all_terms = list()

  jsonRelations['relations'].extend(main_relations)


  jsonEntities = {
      "entities": list()
  }

  jsonEntities['entities'].extend(terms[FIRST_TERM:LAST_TERM])
  jsonEntities['entities'].extend(relation_terms)

  print(jsonEntities)
  print(jsonRelations)

  jsonString.update(jsonEntities)
  jsonString.update(jsonRelations)
  jsonString.update({"Comments": []})

  # Escrita dos JSON
  #jsonFile = 'texts/output.jsonl'
  jsonFile = ''
  jsonFile = 'texts/output_' + str(i) + '-' + str(FIRST_TERM) + '_a_' + str(LAST_TERM) + '.jsonl'

  # linha de geração do arquivo em disco fatiado foi comentada - isso tá funcionando
  # with open(jsonFile, 'w', encoding='utf-8') as json_file:
  #     json.dump(jsonString, json_file, ensure_ascii=False)
  #print(f"Resultados salvos em {jsonFile}")

  if jsonString != '':
    JSONL_COMPLETO.append(jsonString)

  FIRST_TERM = LAST_TERM + 1
  LAST_TERM = LAST_TERM + TERMOS_DIVISAO

  if LAST_TERM > TOTAL_TERMOS:
    LAST_TERM = TOTAL_TERMOS
    j=TOTAL_DIVISAO
  i=i+1
#  Fim do While
if JSONL_COMPLETO != '':
  jsonFile = 'texts/output_' + str(i) + '-' + str(FIRST_TERM) + '_a_' + str(LAST_TERM) + ' - COMPLETO.jsonl'
  #with open(jsonFile, 'w', encoding='utf-8') as json_file:
  #      json.dump(JSONL_COMPLETO, json_file, ensure_ascii=False)
  with open(jsonFile, 'w', encoding='utf-8') as json_file:
    for ddict in JSONL_COMPLETO:
        jout = json.dumps(ddict) + '\n'
        json_file.write(jout)
  print(f"Resultados salvos em {jsonFile}")

{'entities': [{'id': 0, 'label': 'entity', 'start_offset': 0, 'end_offset': 13, 'texto_key': 'ABALROAMENTO'}, {'id': 4, 'label': 'entity', 'start_offset': 230, 'end_offset': 244, 'texto_key': 'ABASTECIMENTO'}, {'id': 11, 'label': 'entity', 'start_offset': 754, 'end_offset': 761, 'texto_key': 'ABICAR'}, {'id': 14, 'label': 'entity', 'start_offset': 941, 'end_offset': 958, 'texto_key': 'ABRIGO ANTIAÉREO'}, {'id': 17, 'label': 'entity', 'start_offset': 1116, 'end_offset': 1128, 'texto_key': 'ACAMPAMENTO'}, {'id': 20, 'label': 'entity', 'start_offset': 1269, 'end_offset': 1283, 'texto_key': 'ACANTONAMENTO'}, {'id': 22, 'label': 'entity', 'start_offset': 1407, 'end_offset': 1412, 'texto_key': 'AÇÃO'}, {'id': 24, 'label': 'entity', 'start_offset': 1537, 'end_offset': 1548, 'texto_key': 'AÇÃO AÉREA'}, {'id': 25, 'label': 'entity', 'start_offset': 1610, 'end_offset': 1629, 'texto_key': 'AÇÃO CÍVICO-SOCIAL'}, {'id': 33, 'label': 'entity', 'start_offset': 2151, 'end_offset': 2168, 'texto_key': '

In [ ]:
# tentando resolver o problema de out of memory
# limpar o cache
import torch
torch.cuda.empty_cache()

## Teste de fatiamento do arquivo


In [ ]:
import math
FIRST_TERM = 0
TERMOS_DIVISAO = 100
LAST_TERM = TERMOS_DIVISAO
TOTAL_TERMOS = 500
TOTAL_DIVISAO=math.floor(TOTAL_TERMOS/TERMOS_DIVISAO)
i=0
while i<=TOTAL_DIVISAO:
    print('Processando dados de ' + str(i) +  ' a ' + str(TOTAL_DIVISAO) + ' - Termos de: ' + str(FIRST_TERM) + ' a ' + str(LAST_TERM))
    FIRST_TERM = LAST_TERM + 1
    LAST_TERM = LAST_TERM + TERMOS_DIVISAO

    if LAST_TERM > TOTAL_TERMOS:
      LAST_TERM = TOTAL_TERMOS
      j=TOTAL_DIVISAO
    i=i+1


Processando dados de 0 a 5 - Termos de: 0 a 100
Processando dados de 1 a 5 - Termos de: 101 a 200
Processando dados de 2 a 5 - Termos de: 201 a 300
Processando dados de 3 a 5 - Termos de: 301 a 400
Processando dados de 4 a 5 - Termos de: 401 a 500
Processando dados de 5 a 5 - Termos de: 501 a 500
